In [1]:
import json
import numpy as np
import re
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [2]:
#add code to download SQuAD

In [3]:
with open('SQuAD/train-v2.0.json') as file:
    train=json.load(file)
# each paragraph is a different topic indicated by the title
# each topic contains a number of different short passages related to the topic, the context
# for each topic there are ~10 question/answer pairs, and an indicator if a question is impossible

In [4]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split(r'(\W+)?', sent) if x.strip()]

In [90]:
train_context=[]
train_question=[]
train_answer=[]
train_new={'context':train_context,'question':train_question,'answer':train_answer}
for j,data in enumerate(train['data']):
    for i,paragraph in enumerate(data['paragraphs']):
        context=tokenize(paragraph['context'])
        for qas in paragraph['qas']:
            #create a dataset with only the answerable questions
            if (qas['is_impossible']==False):
                train_new['context'].append(context)
                train_new['question'].append(tokenize(qas['question']))
                train_new['answer'].append(tokenize(qas['answers'][0]['text']))

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [91]:
print(np.shape(train_new['context']))
train_all=[]
for line in train_new['context']:
    train_all.append(line)
for line in train_new['question']:
    train_all.append(line)
for line in train_new['answer']:
    train_all.append(line)
print(np.shape(train_all))

(86821,)
(260463,)


In [40]:
vocab = set()
for context,question,answer in zip(train_new['context'],train_new['question'],train_new['answer']):
#     print(type(context))
#     print(type(question))
#     print(type(answer))
#     vocab |= set(context+question+[str(answer)])
    vocab |= set(context+question+answer)
vocab = sorted(vocab)
vocab_size = len(vocab) + 1
print(np.shape(vocab))

(96585,)


In [50]:
# word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
#needed for sequence padding
context_maxlen = max(map(len, (x for x, _, _ in zip(train_new['context'],train_new['question'],train_new['answer']))))
question_maxlen = max(map(len, (x for _, x, _ in zip(train_new['context'],train_new['question'],train_new['answer']))))

In [51]:
# print(len(word_idx))
print(context_maxlen)
print(question_maxlen)

844
60


In [10]:
#tfidf
import nltk
# nltk.download('brown')
from nltk.corpus import brown
from collections import Counter
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [11]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [12]:
vocab_size = 5000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(brown.sents())

tfidf = tokenizer.texts_to_matrix(brown.sents(), 'tfidf').T
tfidf /= np.linalg.norm(tfidf, axis= -1, keepdims= True) + 1e-16

In [13]:
sim = lambda w1, w2: tfidf[tokenizer.word_index[w1]] @ tfidf[tokenizer.word_index[w2]]
test = lambda f, ws: f(*[tfidf[tokenizer.word_index[w]] for w in ws])

In [14]:
print(len(brown.sents()))
print(len(tfidf))

57340
5000


In [15]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [44]:
# cbow
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Lambda
from functools import partial

In [45]:
context = 3
seed = 0
dim = 200
vocab_size = 5000

In [47]:
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(brown.sents())

In [48]:
ngrams = [ngram
          for sent in tokenizer.texts_to_sequences(brown.sents())
          for ngram in zip(*[sent[i:] for i in range(context + 1 + context)])]
x = np.array([n[:context] + n[-context:] for n in ngrams])
y = np.array([n[context] for n in ngrams])
del ngrams
print(len(x))
print(len(y))

In [75]:
# ngrams = [ngram
#           for sent in tokenizer.texts_to_sequences(train_all)
#           for ngram in zip(*[sent[i:] for i in range(context + 1 + context)])]
# x = np.array([n[:context] + n[-context:] for n in ngrams])
# y = np.array([n[context] for n in ngrams])
# del ngrams
# print(len(x))
# print(len(y))

9508740
9508740


In [23]:
# np.random.seed(seed)
# np.random.shuffle(x)
# np.random.seed(seed)
# np.random.shuffle(y)

# x_valid = x[:4096]
# y_valid = y[:4096]
# x_train = x[4096:]
# y_train = y[4096:]


# model = Sequential()
# model.add(Embedding(vocab_size, dim, input_length= 2*context))
# model.add(Lambda(partial(K.mean, axis= 1)))
# model.add(Dense(vocab_size, activation= 'softmax')) 
# model.compile(loss= 'sparse_categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
# model.summary()


# model.fit(x_train, y_train, validation_data= (x_valid, y_valid), epochs= 6, batch_size= 256, shuffle= False)


# embed = model.get_weights()[0]

# with open("embeddings1.vec", 'w') as f:
#     print(vocab_size-1, dim, file= f)
#     for i in range(1, vocab_size):
#         print(tokenizer.index_word[i], " ".join(map(str, embed[i])), file= f)

In [83]:
# np.shape(model.get_weights()[2])
with open('embeddings1.vec') as f:
    for i,line in enumerate(f):
        values=line.split()
        print(values[0])
        if i==4:
            break

4999
the
,
.
of


In [ ]:
#fasttext

In [ ]:
#glove